# Librerias

In [31]:
import pymssql        # Conexión a Azure para los datos
import pandas as pd
from pandas_profiling import ProfileReport
from pandasql import sqldf 

# Datos

In [28]:
conn = pymssql.connect(
     server="10khub-server.database.windows.net",
     port="1433",
     user="dbadmin",
     password="P5<`>Qqexw#RS8e[",
     database="hubDB")

strokeDf = pd.read_sql("SELECT * from datahub", conn)

conn.close()

# Etapa 1: Exploración

## Descripción de datos
¿Por qué seleccionamos estos datos?, ¿Qué representan nuestras variables? (Incluir un diccionario, variable: Significado)

In [9]:
strokeDf.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67,0,1,Yes,Private,Urban,228.690002,36.6,formerly smoked,1
1,51676,Female,61,0,0,Yes,Self-employed,Rural,202.210007,N/A,never smoked,1
2,31112,Male,80,0,1,Yes,Private,Rural,105.919998,32.5,never smoked,1
3,60182,Female,49,0,0,Yes,Private,Urban,171.229996,34.4,smokes,1
4,1665,Female,79,1,0,Yes,Self-employed,Rural,174.119995,24,never smoked,1


Escogimos estos datos primero por su aparente limpieza, segundo porque 800, 000 personas al año tienen un accidente cerebrovasculares y más de 140, 000  mueren y de los que sí sobreviven quedan con discapacidades. De acuerdo con cdc.gov, los accidentes son prevenibles. Lo que hizo que nos interesara el dataset para poner nuestro granito de arena en la prevención al decir si alguien puede o no tener un ataque cardiáco.

[Data dada por datathon - Click Aquí](https://docs.google.com/spreadsheets/d/1tjx0q4n3NANh04rfEtl0phkvDOscV1I51EcvZ9-cIG0/edit?usp=sharing)

[Dataset Explicado - Click Aquí](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset)


CDCespanol. (2017, September 6). Prevención de muertes por accidentes cerebrovasculares. Centers for Disease Control and Prevention. https://www.cdc.gov/spanish/signosvitales/accidentes-cerebrovasculares/index.html

In [11]:
strokeDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   int64  
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                5110 non-null   object 
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 479.2+ KB


Cada fila es una persona dada la situación médica, se le nombrará paciente. Se mencionan datos demográficos y sobre su estado de salud tanto en variables categóricas como numericas. La columna "stroke" indica si la el paciente tuvu un accidente cerebrovascular

In [24]:
profile = ProfileReport(strokeDf, title="Stroke EDA Report", explorative=True)
profile.to_file(output_file="Raw_StrokeData_Report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 167.18it/s]


Para explorar un poco más rápido la data se utilizó la librería de Explorative Data Analysis que permite acelerar el entendimiento de las variables que se tienen en nuestro dataset. El reporte se encuentra en [Raw_StrokeData_Report.html](https://github.com/ArmandoDLaRosa/10k-Datathon/blob/main/Raw_StrokeData_Report.html)

In [78]:
# En el Pandas profiling se encontró que se tenía id como campo único,
# pero queríamos ver que tantas filas únicas eran únicas, para considear
# a un id como una persona distinta.
strokeTmp = strokeDf
print("Cantidad Filas Totales ",strokeDf.shape[0], "\nCantidad Filas Únicas  ", strokeTmp.drop_duplicates().shape[0])

Cantidad Filas Totales  5110 
Cantidad Filas Únicas   5110


In [51]:
# En el pandas profiling se encontraron datos aludiendo a personas de 0 años, 
# así que fuimos a investigar si eran una muestra representativa. Pero, solo
# se encontraron que eran un 0.84% del total de datos
pysqldf = lambda q: sqldf(q, globals())

query = """SELECT (count(*)*100/5110.0) as babies_Percent
       FROM strokeDf
       WHERE age < 1"""
display(pysqldf(query))

,babies_Percent
0,0.841487


In [100]:
# Sale la duda de hasta que edad consideran que es un niño en el dataset
query1 = """   SELECT max(age) as Kids_MaxAge
              FROM strokeDf
              WHERE work_type = 'children'"""

query2 = """   SELECT age as YouthAge
              FROM strokeDf
              WHERE age > 10 and age < 21
              GROUP BY age """ 
                      
query3 = """  SELECT distinct age, work_type
              FROM strokeDf
              WHERE age > 16 and age < 21"""        

display(pysqldf(query1))
display(pysqldf(query2))
display(pysqldf(query3))



,Kids_MaxAge
0,16


,YouthAge
0,11
1,12
2,13
3,14
4,15
5,16
6,17
7,18
8,19
9,20


,age,work_type
0,20,Private
1,17,Self-employed
2,18,Private
3,17,Private
4,20,Govt_job
5,18,Self-employed
6,19,Private
7,19,Self-employed
8,19,Never_worked
9,17,Never_worked


## Objetivo
Que queremos saber de los datos

## Selección
¿Están limpios los datos?, ¿cuáles son relevantes?, ¿Cómo preparamos los datos?

## Exploración
Grafica variables vs otras. Dos graficas por integrante de equipo junto una explicación sobre las observaciones.

Usa Watson, Tableau, Power BI para explorar relaciones entre datos y reporta lo que encontraste.

In [ ]:
# Correlacion 
# Dispersión

# Etapa 2: Análisis estadístico
# @Jesus, Adrian

* Histograma
* Media
* Varianza
* Desviación estándar
* Simetría
* Curtosis 

Interpretación de los estadísticos descriptivos sobre dichas variables, ofrece una explicación sobre la varianza, media, desviación estandar.
Estas son hipotesis sobre los datos. Recuerda redactar esto precisamente como hipotesis, y basarte en los números que obtuviste en el análisis. Por ultimo, para cada una de las variables incluye también tu inferencia sobre que distribución de probabilidad y explica el comportamiento de la variable.

Basandonos en la forma del histograma, pesamos que esta variable podría tener una distribución nomal de la información y se encuentra cesgado hacia la derecha por lo que podriamos asumir que la variable tiene una mayor cantidad de información en este rango de numeros.

# Etapa 3: Modelación

## Modela los datos con alguna tecnica.
* Teoría de Colas
* Leyes operativas
* K-Medias
* Redes neuronales
* Redes neuronales profundas
* Redes bayesianas
* Árboles de decisión 
* Bosques Aleatorios
* PCAs
* Algoritmos genéticos
* SVM
* Regresión

### Se utilizaran
- PCAs  @Jesus
- K-Medias @Angel
- Árboles de decisión @Armando, Shap, Pipeline
- Regresión logistica binaria @Adrian
- (Quiza Redes Neuronales) @ Todos

Reporta la tecnica, el modelo, sus ajustes y tus resultados.

Pro-tip: usa dos o más métodos para agrupar o predecir y comparalos para un mejor análisis.

# Etapa 4: Conclusiones

Presenta una lista de insights relevantes de acuerdo al objetivo (problema) que declaraste al principio de este documento, fundamentada en las observaciones, modelado y resultados de la simulación.
Un insight es encontrar una relación entre los datos que no es obvia, que puede ayudar al cliente a resolver problemas o a mejorar algun aspecto de su negocio (moneratio, inventario, producción…etc.).